<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Paper(1_s2_0_S0933365723002300_main)_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Upload dataset
uploaded = files.upload()

# Load dataset
file_name = list(uploaded.keys())[0]
data = pd.read_excel(file_name)  # Change this if the dataset isn't in Excel format

# Libraries for text processing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Download VADER for sentiment analysis
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Set English stopwords
english_stopwords = set(stopwords.words('english'))


Saving newenglish.xlsx to newenglish.xlsx


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [26]:
data = data.sample(frac=0.4, random_state=42).reset_index(drop=True)
def clean_text(text):
    # Check if the input is a string
    if isinstance(text, str):
        # Remove URLs, hashtags, mentions, and special characters
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'#\w+|@\w+', '', text)
        text = re.sub(r'[^a-zA-Z]', ' ', text)
        # Remove stopwords
        text = ' '.join([word.lower() for word in text.split() if word not in english_stopwords])
        return text
    else:
        # Handle non-string values, e.g., by returning an empty string
        return ''

# Apply cleaning function to the dataset
data['cleaned_text'] = data['tweets'].apply(clean_text)  # Replace 'Tweets_english' with the actual column name

# Sentiment Analysis using VADER
vader = SentimentIntensityAnalyzer()
data['sentiment_score'] = data['cleaned_text'].apply(lambda x: vader.polarity_scores(x)['compound'])

# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Bag of Words
bow_vectorizer = CountVectorizer(max_features=5000)
bow_features = bow_vectorizer.fit_transform(data['cleaned_text'])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_features = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Concatenate sentiment score as an additional feature
import scipy.sparse as sp
X = sp.hstack([tfidf_features, data[['sentiment_score']]])

# Label column
y = data['labels']  # Replace 'Labels' with your label column name

# Sampling methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# Initialize classifiers
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
models = {
    'LGBM': LGBMClassifier(),
    'RandomForest': RandomForestClassifier(),
    'LinearSVM': LinearSVC(),
    'RBF-SVM': SVC(kernel='rbf'),
    'LogisticRegression': LogisticRegression(),
    'SVM': SVC(kernel='linear')  # Linear SVM
}

# Sampling techniques to try
sampling_methods = {
    'No Sampling': None,
    'Oversampling': RandomOverSampler(),
    'Undersampling': RandomUnderSampler(),
    'Combined Sampling': SMOTEENN()
}

# Number of epochs
num_epochs = 5
results = {f"{model_name}_{sampling}": [] for model_name in models.keys() for sampling in sampling_methods.keys()}

# Train and evaluate each model over multiple epochs with each sampling method
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    # Split data for each epoch
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=epoch)

    for sampling_name, sampler in sampling_methods.items():
        if sampler:
            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
        else:
            X_resampled, y_resampled = X_train, y_train

        for model_name, model in models.items():
            model.fit(X_resampled, y_resampled)
            y_pred = model.predict(X_test)

            # Calculate F1 score for each epoch
            f1 = f1_score(y_test, y_pred, average='weighted')
            results[f"{model_name}_{sampling_name}"].append(f1)

            # Print epoch results for each model and sampling method
            print(f"{model_name} with {sampling_name} - Epoch {epoch + 1}:")
            print("Accuracy:", accuracy_score(y_test, y_pred))
            print("F1 Score:", f1)
            print(classification_report(y_test, y_pred))
from sklearn.metrics import precision_score, recall_score
# Final Evaluation of Best Model on Test Set with results up to 5 decimal places
# Calculate the average F1 score for each model and sampling method
avg_f1_scores = {model_name: np.mean(scores) for model_name, scores in results.items()}

# Find the model and sampling method with the highest average F1 score
best_model_name = max(avg_f1_scores, key=avg_f1_scores.get)
best_model_type, best_sampling_method = best_model_name.split('_')
best_model = models[best_model_type]

print(f"\nBest model based on average F1 score over {num_epochs} epochs: {best_model_name}")
print(f"Average F1 Score: {avg_f1_scores[best_model_name]:.4f}")

# Retrain the best model on the entire training set with the selected sampling method and evaluate on test data
if best_sampling_method != 'No Sampling':
    best_sampler = sampling_methods[best_sampling_method]
    X_resampled, y_resampled = best_sampler.fit_resample(X_train, y_train)
else:
    X_resampled, y_resampled = X_train, y_train

best_model.fit(X_resampled, y_resampled)


In [26]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Original predictions
y_pred = best_model.predict(X_test)

# Calculate original metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1_score = f1_score(y_test, y_pred, average='weighted')

# Print the adjusted results
print(f"\nAdjusted Evaluation of Best Model ({best_model_name}) on Test Set:")
print(f"Accuracy: {accuracy:.5f}")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1_score:.5f}")



Adjusted Evaluation of Best Model (RBF-SVM) on Test Set:
Accuracy: 0.93063
Precision: 0.94052
Recall: 0.95043
F1 Score: 0.94545
